In [1]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.utils import img_to_array
from keras.models import load_model
import tensorflow as tf
from PIL import Image
import numpy as np
from io import BytesIO
import io
import joblib
from fastapi import FastAPI,UploadFile,File
from pydantic import BaseModel
import pickle
import uvicorn
import json
import nest_asyncio


In [2]:
! pip install pyngrok

     ------------------------------------ 761.3/761.3 kB 302.4 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 151.7/151.7 kB 1.1 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19793 sha256=e765a0505b10240aaac748af7548e9651fe51b0f895665fed06a5a97ae6450aa
  Stored in directory: c:\users\marwan gamal\appdata\local\pip\cache\wheels\1a\b2\89\c70c614777a68db33b539d5b3ba68d22bda56c0aa827fe1a3e
Successfully built pyngrok


In [2]:
app = FastAPI()

In [3]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [4]:
def read_imagefile(file) -> Image.Image:
    image = Image.open(BytesIO(file))
    return image

______

### Skin cancer

_______

In [5]:
@app.post("/skin_cancer_predict")

async def predict_api(file: UploadFile = File(...)):
    extension = file.filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not extension:
        return "Image must be jpg or png format!"
    
    image = read_imagefile(await file.read())
    # image = Image.open(image)
    if image.mode != "RGB":
        image = image.convert("RGB")

    # resize the input image and preprocess it
    image = image.resize((28,28))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    
    model_Skin= tf.keras.models.load_model("skin_cancer_model.sav")
    
    label_mapping = {
    0: 'nv',
    1: 'mel',
    2: 'bkl',
    3: 'bcc',
    4: 'akiec',
    5: 'vasc',
    6: 'df'}

    lesion_type_dict = {
        'nv': 'Melanocytic nevi',
        'mel': 'Melanoma',
        'bkl': 'Benign keratosis-like lesions',
        'bcc': 'Basal cell carcinoma',
        'akiec': 'Actinic keratoses',
        'vasc': 'Vascular lesions',
        'df': 'Dermatofibroma'}

    mixed = {
        0: 'Melanocytic nevi',
        1: 'Melanoma ',
        2: 'Benign keratosis-like lesions',
        3: 'Basal cell carcinoma',
        4: 'Actinic keratoses',
        5: 'Vascular lesions',
        6: 'Dermatofibroma'}
    
    return lesion_type_dict[label_mapping[np.argmax(model_Skin.predict(image))]]

__________

### Diabetes

______

In [6]:
class model_input_Diabetes(BaseModel):
    
    Age : int
    Gender: int
    Polydipsia : int
    Polyuria : int
    Sudden_Weight_Loss : int
    Alopecea : int


In [11]:
model_diabetes =joblib.load('RfModify_model_diabetes.hdf5')

In [12]:
@app.post('/diabetes_prediction')
def diabetes_predd(input_parameters : model_input_Diabetes):
    
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)
    
    Age = input_dictionary['age']
    Gender = input_dictionary['gender']
    Polydipsia = input_dictionary['Polydipsia']
    Polyuria = input_dictionary['Polyuria']
    Sudden_Weight_Loss = input_dictionary['sudden_weight_loss']
    Alopecea = input_dictionary['Alopecia']
    
    
    input_list = [Age, Gender, Polydipsia, Polyuria, Sudden_Weight_Loss, Alopecea]
    
    prediction = model_diabetes.predict([input_list])
    
    if (prediction[0] == 0):
        return 'Negative'
    else:
        return 'Positive'

___________________

### Chest

_____

In [8]:
@app.post("/chest_predict")

async def predict_api(file: UploadFile = File(...)):
    extension = file.filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not extension:
        return "Image must be jpg or png format!"
    
    image = read_imagefile(await file.read())
    
    image = image.resize((128,128))
    image = image.convert("L")
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    model_Xray= tf.keras.models.load_model('x-ray_chest87.sav')
    
    preds = model_Xray.predict(image)
    
    all_labels = [
     'Atelectasis',
     'Cardiomegaly',
     'Consolidation',
     'Edema',
     'Effusion',
     'Emphysema',
     'Fibrosis',
     'Infiltration',
     'Mass',
     'Nodule',
     'Pleural_Thickening',
     'Pneumonia',
     'Pneumothorax']
    
    l={}
    for n_class, p_score in zip(all_labels, preds):
        l[n_class[:]]=p_score*100

    return max(l,key=l.get)

_____________________

### Brain tumor

______

In [9]:
@app.post("/brain_tumor_predict")

async def predict_api(file: UploadFile = File(...)):
    extension = file.filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not extension:
        return "Image must be jpg or png format!"
    
    image = read_imagefile(await file.read())
    
    image = image.resize((150, 150))
    image = image.convert("L")
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    model_Brain =tf.keras.models.load_model('Brain_Tumor.hdf5')
    
    #Brain Tumor labels
    label_mapping_brain = {
        0: "glioma_tumor",
        2:"meningioma_tumor",
        1:"no_tumor",
        3:"pituitary_tumor"}

    all_labels_brain = {
     "glioma_tumor":'Giloma Tumor',
     "no_tumor" :'No Tumor',
     "meningioma_tumor" :'Meningioma Tumor',
     "pituitary_tumor" :'Pituitary Tumor'}
    
    return all_labels_brain[label_mapping_brain[np.argmax(model_Brain.predict(image))]]

In [10]:
#ngrok_tunnel=ngrok.connect(8000)
#print('public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [15628]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50176 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50176 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15628]
